In [35]:
import tqdm as tqdm
import os
import torch
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import datetime
from torchvision import transforms  
from torch.optim import lr_scheduler

class TreeNode(object):
     def __init__(self, n_eta, split_attribute_M, split_value,  y_pred,
                        y_avg, posterior_var, posterior_mean,
                        left=None, right=None):
          self.n_eta = n_eta
          self.split_attribute_M = split_attribute_M   # internal node
          self.split_value = split_value  # internal node
          self.y_pred = y_pred  # leaf
          self.y_avg = y_avg # leaf
          self.posterior_var = posterior_var # leaf
          self.posterior_mean = posterior_mean  #leaf
          self.left = left
          self.right = right

class NN(nn.Module):
    def __init__(self,dim):
        super(NN, self).__init__()
        self.lin = nn.Sequential(
            nn.Linear(dim, 32),
            nn.Linear(32, dim),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.lin(input)

    def set_connections(self,node):
        pass

In [30]:
trees = []
with open('mydata.txt', 'r') as fp:
    for line in fp.readlines():
        vals = iter(line.split())

        def helper():
            val = next(vals)
            if val == '#':ele
                return None
            elems = val.split("@")
            node = TreeNode(float(elems[0]), float(elems[1]), 
                            float(elems[2]), float(elems[3]), 
                            float(elems[4]),float(elems[5]), float(elems[6]))
            node.left = helper()
            node.right = helper()
            return node
        trees.append(helper())
        